# Ax for AutoML with scikit-learn
Automated machine learning (AutoML) encompasses a large class of problems related to automating time-consuming and labor-intensive aspects of developing ML models.
Adaptive experimentation is a natural fit for solving many AutoML tasks, which are often iterative in nature and can involve many expensive trial evaluations.

In this tutorial we will use Ax for hyperparameter optimization (HPO), a common AutoML task in which a model's hyperparameters are adjusted to improve model performance.
Hyperparameters refer to the parameters which are set prior to model training or fitting, rather than parameters being learned from data.
Traditionally, ML engineers use a combination of domain knowledge, intuition, and manual experimentation comparing many models with different hyperparameter configurations to determine good hyperparameters.
As the number of hyperparameters grows and as models become more expensive to train and evaluate sample efficient aproaches to experimentation like Bayesian optimization become increasingly valuable.

In this tutorial we will train an `SGDClassifier` from the popular [scikit-learn](https://scikit-learn.org/) library to recognize handwritten digits and tune the model's hyperparameters to improve its performance.
You can read more about the `SGDClassifier` model in their example [here](https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html), which this tutorial is largely based on.
This tutorial will incorporate many advanced features in Ax to demonstrate how they can be applied on complex engineering challenges in a real-world setting.

## Learning Objectives
- Understand how Ax can be used for HPO tasks
- Use complex optimization configurations like multiple objectives and outcome constraints to achieve nuanced real-world goals
- Use early stopping to save experimentation resources
- Analyze the results of the optimization

## Prerequisites
- Familiarity with [scikit-learn](https://scikit-learn.org/) and basic machine learning concepts
- Understanding of [adaptive experimentation](../../intro-to-ae) and [Bayesian optimization](../../intro-to-bo)
- [Ask-tell Optimization of Python Functions with early stopping](../early_stopping)

## Step 1: Import Necessary Modules

First, ensure you have all the necessary imports:

In [ ]:
import time

import matplotlib.pyplot as plt

import numpy as np

import sklearn.datasets
import sklearn.linear_model
import sklearn.model_selection

from ax.api.client import Client
from ax.api.configs import ChoiceParameterConfig, RangeParameterConfig

from pyre_extensions import assert_is_instance

## Step 1.1: Understanding the baseline performance of `SGDClassifier`
Before we begin HPO, let's understand the task and the performance of `SGDClassifier` with its default hyperparameters.
The following code is largely adapted from the example on scikit-learn's webiste [here](https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html).



In [ ]:
# Load the digits dataset and display the first 4 images to demonstrate
digits = sklearn.datasets.load_digits()
classes = list(set(digits.target))

_, axes = plt.subplots(nrows=1, ncols=4, figsize=(10, 3))
for ax, image, label in zip(axes, digits.images, digits.target):
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
    ax.set_title("Training: %i" % label)

In [ ]:
# Split the data into a training set and a validation set
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    digits.data, digits.target, test_size=0.20, random_state=0
)

# Instantiate a SGDClassifier with default hyperparameters
clf = sklearn.linear_model.SGDClassifier()

# Train the classifier on the training set using 10 batches.
# Also time the training.
n_batches = 10
batch_size = X_train.shape[0] // n_batches
start_time = time.monotonic()
for epoch in range(20):
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        X_batch = X_train[start:end]
        y_batch = y_train[start:end]

        clf.partial_fit(X_batch, y_batch, classes=classes)

training_time = time.monotonic() - start_time

# Evaluate the classifier on the validation set
score = clf.score(X_test, y_test)
score, training_time

The model performs well, but let's see if we can improve performance by tuning the hyperparameters.

## Step 2: Initialize the Client
As always, the first step in running our adaptive experiment with Ax is to create an instance of the `Client` to manage the state of your experiment.

In [ ]:
client = Client()

## Step 3: Configure the Experiment

The `Client` expects a series of `Config`s which define how the experiment will be run.
We'll set this up the same way as we did in our previous tutorial.

Our current task is to tune the hyperparameters of an scikit-learn's [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html).
These parameters control aspects of the model's training process and configuring them can have dramatic effects on the model's ability to correctly classify inputs.
A full list of this model's hyperparameters and appropriate values are available in the library's documentation.
In this tutorial we will tune the following hyperparameters:
* **loss:** The loss function to be used
* **penalty:** The penalty (aka regularization term) to be used
* **learning_rate:** The learning rate schedule
* **alpha:** Constant that multiplies the regularization term. The higher the value, the stronger the regularization
* **eta0**: The learning rate for training. In this example we will use a constant learning rate schedule
* **batch_size**: A training parameter which controls how many examples are shown during a single call to `partial_fit`
 
You will notice some hyperparameters are continuous ranges, some are discrete ranges, and some are categorical choices; Ax is able to handle all of these types of parameters via its `RangeParameterConfig` and `ChoiceParameterConfig` classes.

In [ ]:
# Configure and experiment with the desired parameters
client.configure_experiment(
    parameters=[
        ChoiceParameterConfig(
            name="loss",
            parameter_type="str",
            values=[
                "hinge",
                "log_loss",
                "squared_hinge",
                "modified_huber",
                "perceptron",
            ],
            is_ordered=False,
        ),
        ChoiceParameterConfig(
            name="penalty",
            parameter_type="str",
            values=["l1", "l2", "elasticnet"],
            is_ordered=False,
        ),
        ChoiceParameterConfig(
            name="learning_rate",
            parameter_type="str",
            values=["constant", "optimal", "invscaling", "adaptive"],
            is_ordered=False,
        ),
        RangeParameterConfig(
            name="alpha",
            bounds=(1e-8, 100),
            parameter_type="float",
            scaling="log",  # Sample this parameter in log transformed space
        ),
        RangeParameterConfig(
            name="eta0",
            bounds=(1e-8, 1),
            parameter_type="float",
            scaling="log",
        ),
        RangeParameterConfig(
            name="batch_size",
            bounds=(5, 500),
            parameter_type="int",
        ),
    ]
)

## Step 4: Configure Optimization
Now, we must set up the optimization objective in `Client`, where `objective` is a string that specifies which metric we would like to optimize and the direction (higher or lower) that is considered optimal.

In our example we want to consider both performance and computational cost implications of hyperparameter modifications.
`scikit-learn` models use a function called `score` to report the mean accuracy of the model, and in our optimization we should seek to maximize this value.
Since model training can be a very expensive process, especially for large models, this can represent a significant cost.

Let's configure Ax to maximize score while minimizing training time.
We call this a multi-objective optimization, and rather than returning a single best parameterization we return a Pareto frontier of points which represent optimal tradeoffs between all metrics present.
Multi-objective optimization is useful for competing metrics where a gain in one metric may represent a regression in the other.

In these settings we can also specify outcome constraints, which indicate that if a metric result falls outside of the specified threshold we are not interested in any result, regardless of the wins observed in any other metric.
For a concrete example, imagine Ax finding a parameterization that trains in no time at all but has an score no better than if the model were guessing at random.

For this toy example let's configure Ax to maximize score and minimize training time, but avoid any hyperparameter configurations that result in a mean accuracy score of less than 85% or a training time greater than 2 seconds.

In [ ]:
client.configure_optimization(
    objective="score, -training_time",
    outcome_constraints=["score >= 0.85", "training_time <= 2"],
)

## Step 5: Run Trials with early stopping
Before we begin our Bayesian optimization loop, we can attach the data we collected from triaing `SGDClassifier` with default hyperparameters.
This will give our experiment a head start by providing a datapoint to our surrogate model.
Because these are the default settings provided by `scikit-learn`, it's likely they will be pretty good and will provide the optimization with a promising start.
It is always advantageous to attach any existing data to an experiment to improve performance.


In [ ]:
trial_index = client.attach_baseline(
    parameters={
        "loss": clf.loss,
        "penalty": clf.penalty,
        "alpha": clf.alpha,
        "learning_rate": clf.learning_rate,
        "eta0": clf.eta0
        + 1e-8,  # Default eta is 0.0, so add a small value to avoid division by zero
        "batch_size": batch_size,
    }
)

client.complete_trial(
    trial_index=trial_index,
    raw_data={"score": score, "training_time": training_time},
)

After attaching the initial trial, we will begin the experimentation loop by writing a for loop to execute our full experimentation budget of 30 trials.
In each iteration we will ask Ax for the next trials (in this case just one), then instantiate an `SGDClassifier` with the suggested hyperparameters.
Next we will define two inner loops to perform minibatch training, in which we divide the train set into a number of smaller batches and train one epoch of stochastic gradient descent at a time.
After each epoch we will report the score and the time.

Because training machine learning models is expensive, we will utilize Ax's early stopping functionality to kill trials unlikely to produce optimal results before they have been completed.
After data has been attached we will ask the `Client` whether or not we should stop the trial, and if it advises us to do so we will report it early stopped and exit out of the training loop.
By early stopping, we proactively save compute without regressing optimization performance.

In [ ]:
for _ in range(20): # Run 20 rounds of 1 trial each
    trials = client.get_next_trials(max_trials=1)
    for trial_index, parameters in trials.items():
        clf = sklearn.linear_model.SGDClassifier(
            loss=parameters["loss"],
            penalty=parameters["penalty"],
            alpha=parameters["alpha"],
            learning_rate=parameters["learning_rate"],
            eta0=parameters["eta0"],
        )

        batch_size = assert_is_instance(parameters["batch_size"], int)
        n_batches = X_train.shape[0] // batch_size

        # Create a flag to track whether the trial was stopped early
        is_early_stopped = False

        start_time = time.monotonic()
        for epoch in range(20):
            for i in range(n_batches):
                # Calculate the minibatch and fit the model
                start = i * batch_size
                end = start + batch_size

                X_batch = X_train[start:end]
                y_batch = y_train[start:end]

                clf.partial_fit(X_batch, y_batch, classes=classes)

            # Evaluate the model and report the score back to Ax
            score = clf.score(X_test, y_test)

            client.attach_data(
                trial_index=trial_index,
                raw_data={"score": score, "training_time": time.monotonic() - start_time},
                progression=epoch + 1,
            )

            # If the trial is underperforming, stop it
            if client.should_stop_trial_early(trial_index=trial_index):
                is_early_stopped = True
                client.mark_trial_early_stopped(trial_index=trial_index)
                break

        if is_early_stopped:
            break


        if not is_early_stopped:
            client.complete_trial(trial_index=trial_index)


## Step 6: Analyze Results

After running trials, you can analyze the results.
Most commonly this means extracting the parameterization from the best performing trial you conducted.

Since we are optimizing multiple objectives, rather than a single best point we want to get the Pareto frontier -- the set of points that presents optimal tradeoffs between maximizing score and minimizing training time.

In [ ]:
frontier = client.get_pareto_frontier()

# Frontier is a list of tuples, where each tuple contains the parameters, the metric readings, the trial index, and the arm name for a point on the Pareto frontier
for parameters, metrics, trial_index, arm_name in frontier:
    print(f"Trial {trial_index} with {parameters=} and {metrics=}\n")

## Step 7: Compute Analyses

Ax can also produce a number of analyses to help interpret the results of the experiment via `client.compute_analyses`.
Users can manually select which analyses to run, or can allow Ax to select which would be most relevant.
In this case Ax selects the following:
* **Arm Effects Plots** show the metric value for each [arm](https://ax.dev/docs/next/glossary#arm) on the experiment. Ax produces one plot using values from its internal surrogate model (this can be helpful for seeing the true effect of an arm when evaluations are noisy) and another using the raw metric values as observed.
* **Scatter Plot** shows the effects of each trial on two metrics, and is useful for understanding the trade-off between the two outcomes
* **Summary** lists all trials generated along with their parameterizations, observations, and miscellaneous metadata
* **Sensitivity Analysis Plot** shows which parameters have the largest affect on the objective using [Sobol Indicies](https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis)
* **Slice Plot** shows how the model predicts a single parameter effects the objective along with a confidence interval
* **Contour Plot** shows how the model predicts a pair of parameters effects the objective as a 2D surface
* **Cross Validation** helps to visualize how well the surrogate model is able to predict out of sample points 

In [ ]:
# display=True instructs Ax to sort then render the resulting analyses
cards = client.compute_analyses(display=True)

## Conclusion

This tutorial demonstates Ax's ability to solve AutoML tasks with in a resource efficient manor.
We configured a complex optimization which captures the nuanced goals of the experiment and utilized early stopping to save resources by killing training runs unlikely to produce optimal results.

While this tutorial shows how to use Ax for HPO on an `SGDClassifier`, the same techniques can be used for many different AutoML tasks such as feature selection, architecture search, and more.